This notebook performs Singular Value Decomposition on Attention Matrices of T5 Base And creates replaces original weight matrices by their Low Rank Approximation. We are able to **reduce parameters by 25-34% (model size)** with allmost no impact on model's performance (Samsum summarization dataset)

In [1]:
!pip install rouge
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git
# install additional dependencies needed for training
!pip install rouge-score tensorboard py7zr
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 40.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which i

In [2]:
from torch import nn
from dataclasses import dataclass
from torch.nn import functional as F
import copy
import torch

In [3]:
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer

# huggingface hub model id
model_id_t5_base = "google/flan-t5-base"
#model_id_t5_large = "google/flan-t5-large"

# load model from the hub
model_t5_base = AutoModelForSeq2SeqLM.from_pretrained(model_id_t5_base,  device_map="auto")
#model_t5_large = AutoModelForSeq2SeqLM.from_pretrained(model_id_t5_large,  device_map="auto")

#print_trainable_parameters(model)
tokenizer = AutoTokenizer.from_pretrained(model_id_t5_base)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

# Create Model With Low Rank Weight Matrices

In [4]:
@dataclass
class LowRankConfig:
    rank:int
    target_modules: list[str]

In [5]:
#low rank decomposition of SelfAttention Key, Query and Value Matrices
config = LowRankConfig(
    rank= 384,
    target_modules=["k", "q", "v"]
)

In [6]:
class LowRankLayer(nn.Module):
    """given a linear layer find low rank decomposition"""
    def __init__(self, rank, full_rank_layer):
        super().__init__()
        self.rank = rank
        U, S, Vh = torch.linalg.svd(full_rank_layer.weight)
        S_diag = torch.diag(S)
        self.U = U[:, :self.rank]
        self.S = S_diag[:self.rank, :self.rank]
        self.Vh = Vh[:self.rank, :]

    def forward(self, x):
        aprox_weight_matrix = self.U @ self.S @ self.Vh
        output = F.linear(x, aprox_weight_matrix)
        return output


Helper Functions

In [7]:
#find the module that ends target suffix
def get_submodules(model, key):
    parent = model.get_submodule(".".join(key.split(".")[:-1]))
    target_name = key.split(".")[-1]
    target = model.get_submodule(key)
    return parent, target, target_name

# this function replaces a target layer with low rank layer
def recursive_setattr(obj, attr, value):
    attr = attr.split('.', 1)
    if len(attr) == 1:
        setattr(obj, attr[0], value)
    else:
        recursive_setattr(getattr(obj, attr[0]), attr[1], value)


**create low rank replica of original model (model_t5_base)**

In [8]:
#create a copy of original model
model_t5_base_lr = copy.deepcopy(model_t5_base)


# SVD: Low rank decomposition of SelfAttention Key, Query and Value Matrices

In [9]:
for key, module in model_t5_base.named_modules():
    target_module_found = any(key.endswith("." + target_key) for target_key in config.target_modules)
    if target_module_found:
        low_rank_layer = LowRankLayer(config.rank, module)
        #replace target layer with low rank layer
        recursive_setattr(model_t5_base_lr, key, low_rank_layer)

In [10]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

***Number of Parameters : Original Model vs Low Rank Model ***

In [11]:
print_trainable_parameters(model_t5_base), print_trainable_parameters(model_t5_base_lr)

trainable params: 247577856 || all params: 247577856 || trainable%: 100.0
trainable params: 183876864 || all params: 183876864 || trainable%: 100.0


(None, None)

**Model Size Reduction (945M to 702M) ~ 25%**

In [12]:
model_t5_base.save_pretrained("model_t5_base", from_pt=True)
model_t5_base_lr.save_pretrained("model_t5_base_lr", from_pt=True)


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2525: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2525: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(


In [13]:
!ls -lh model_t5_base/pytorch_model.bin

ls: cannot access 'model_t5_base/pytorch_model.bin': No such file or directory


In [14]:
!ls -lh model_t5_base_lr/pytorch_model.bin

ls: cannot access 'model_t5_base_lr/pytorch_model.bin': No such file or directory


**Looking into the layers**

In [15]:
model_t5_base_lr

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): LowRankLayer()
              (k): LowRankLayer()
              (v): LowRankLayer()
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=768, bias=False)
              (dropout): Dropout(p=0.1, inplace=False)
      

In [16]:
model_t5_base_lr.decoder.block[11].layer[0].SelfAttention.q

LowRankLayer()

In [17]:
model_t5_base_lr.decoder.block[11].layer[0].SelfAttention.q.U.shape,   model_t5_base_lr.decoder.block[11].layer[0].SelfAttention.q.S.shape, model_t5_base_lr.decoder.block[11].layer[0].SelfAttention.q.Vh.shape

(torch.Size([768, 384]), torch.Size([384, 384]), torch.Size([384, 768]))

# Projecting Random Vector On Original SelfAttention Matrix vs its Low Rank Aproximation

In [18]:
#low rank approximation of model_t5_base.encoder.block[0].layer[0].SelfAttention.q
# 768 to 384 dim reduction
query_attention_layer = model_t5_base.encoder.block[0].layer[0].SelfAttention.q
low_rank_query_attention_layer = LowRankLayer(384, model_t5_base.encoder.block[0].layer[0].SelfAttention.q)

In [19]:
random_vector = torch.rand(768)
low_rank_projection = low_rank_query_attention_layer(random_vector)
original_projection = query_attention_layer(random_vector)

**Cosine Distance Similarity**

In [20]:
cosine_sim = torch.nn.CosineSimilarity(dim=0)
cosine_sim(low_rank_projection, original_projection)

tensor(0.9667, grad_fn=<SumBackward1>)

**0.9663 Cosine Similarity between random vector projection on original vs low rank approximation of query weight matrix**

# Evaluation

In [21]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("samsum")
metric = load_metric("rouge")

The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

<ipython-input-21-a2ef1525156b>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [22]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [23]:
raw_datasets["test"]

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 819
})

**Tokenize Test Dataset**

In [24]:
prefix = "Summarize: "
max_input_length = 512
max_target_length = 64

def preprocess_function(examples):
  # encode the documents
  dialogues = examples['dialogue']
  summaries = examples['summary']

  inputs = [prefix + dialogue for dialogue in dialogues]
  model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

  # encode the summaries
  labels = tokenizer(summaries, max_length=max_target_length, padding="max_length", truncation=True).input_ids

  # important: we need to replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)

  model_inputs["labels"] = labels_with_ignore_index

  return model_inputs

In [25]:
tokenized_datasets = raw_datasets["test"].map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "id"])


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [26]:
len(tokenized_datasets)

819

In [27]:
tokenized_datasets

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 819
})

In [28]:
!pip install nltk

In [29]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 691.0 kB/s eta 0:00:00


In [30]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


***`Evaluate google/flan-t5-base`***

In [31]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq


In [32]:

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model_t5_base,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


This is a dummy Trainer object, it will be only used for evaluation

In [33]:

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir="dummy",
    predict_with_generate=True,
)
trainer = Seq2SeqTrainer(
    model=model_t5_base,
    args=training_args,
    data_collator=data_collator,
    #train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_datasets,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.evaluate()


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1248: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


**Inference**

In [ ]:
raw_datasets["test"][0]

In [ ]:
formatted_query = "Summarize: " + raw_datasets["test"][0]["dialogue"]
tokenized_text = tokenizer(formatted_query, truncation=True, return_tensors='pt')

generated_ids = model_t5_base.generate(**tokenized_text)
predictions = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
predictions

**Evaluate Compressed flan-t5-base**

In [ ]:
label_pad_token_id = -100
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model_t5_base_lr,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir="dummy",
    predict_with_generate=True,
)
trainer = Seq2SeqTrainer(
    model=model_t5_base_lr,
    args=training_args,
    data_collator=data_collator,
    #train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_datasets,
    compute_metrics=compute_metrics,
)
trainer.evaluate()


**Inference**

In [ ]:
raw_datasets["test"][0]

In [ ]:
formatted_query = "Summarize: " + raw_datasets["test"][0]["dialogue"]
tokenized_text = tokenizer(formatted_query, truncation=True, return_tensors='pt')

generated_ids = model_t5_base_lr.generate(**tokenized_text)
predictions = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
predictions

# Compression V2: Low Rank Approximation Decomposition of SelfAttention Key, Query, Value and Output Matrices

In [ ]:
#low rank decomposition of SelfAttention Key, Query and Value Matrices
config = LowRankConfig(
    rank= 384,
    target_modules=["k", "q", "v", "o"]
)

In [ ]:
#create a copy of original model
model_t5_base_lr_v2 = copy.deepcopy(model_t5_base)


**Create low rank aprox layers**

In [ ]:
for key, module in model_t5_base.named_modules():
    target_module_found = any(key.endswith("." + target_key) for target_key in config.target_modules)
    if target_module_found:
        low_rank_layer = LowRankLayer(config.rank, module)
        #replace target layer with low rank layer
        recursive_setattr(model_t5_base_lr_v2, key, low_rank_layer)

In [ ]:
print_trainable_parameters(model_t5_base), print_trainable_parameters(model_t5_base_lr_v2)

**34.31% Compression in Model size**

# Evaluating compressed model on Samsum Summarization Dataset

In [ ]:
# label_pad_token_id = -100
# data_collator_3 = DataCollatorForSeq2Seq(
#     tokenizer,
#     model=model_t5_base_lr_v2,
#     label_pad_token_id=label_pad_token_id,
#     pad_to_multiple_of=8
# )

# Define training args
training_args_3 = Seq2SeqTrainingArguments(
    output_dir="dummy",
    predict_with_generate=True,
)
trainer_3 = Seq2SeqTrainer(
    model=model_t5_base_lr_v2,
    args=training_args_3,
    # data_collator=data_collator_3,
    #train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_datasets,
    compute_metrics=compute_metrics,
)
trainer_3.evaluate()